# 04_dashboard_publisher
Programmatically create/update Databricks SQL dashboards using databricks-sdk.

In [ ]:
# Install SDK if required: %pip install databricks-sdk
import json
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.sql import CreateDashboardRequest, CreateVisualizationRequest

w = WorkspaceClient()
with open('/dbfs/FileStore/report_metadata/report_definitions.json','r') as f:
    reports = json.load(f)

dashboard_map = {}
for rpt in reports:
    dash_name = rpt.get('dashboard','Reports')
    if dash_name not in dashboard_map:
        created = w.dashboards.create(CreateDashboardRequest(name=dash_name))
        dashboard_map[dash_name] = created.id
    q = w.queries.create(name=f"{rpt['report_name']}_query", query=f"SELECT * FROM finance.kyc_ml.{rpt['report_name']}")
    viz = w.visualizations.create(CreateVisualizationRequest(query_id=q.id, type='table', name=rpt.get('widget_title', rpt['report_name'])))
    w.dashboard_widgets.create(dashboard_id=dashboard_map[dash_name], visualization_id=viz.id)

print('Dashboards created/updated:', dashboard_map)